## 코루틴과 yield 표현식

In [89]:
def receiver():
    print("Ready to receive")
    while True:
        n = yield
        print("Got %s" % n)

In [90]:
r = receiver()
next(r)

Ready to receive


In [92]:
r.send(1)

Got 1


In [93]:
r.send(2)

Got 2


In [94]:
r.send("Hello")

Got Hello


## 코루틴과 데코레이터

In [101]:
def coroutine(func):
    def start(*args, **kwargs):
        g = func(*args, **kwargs)
        next(g)
        return g
    return start

@coroutine
def receiver():
    print("Ready to receive")
    while True:
        n = (yield)
        print("Got %s" % n)

In [102]:
r = receiver()
r.send("Hello World")

Ready to receive
Got Hello World


In [104]:
r.close()
r.send(4)

StopIteration: 

## 생성기와 코루틴의 활용 

In [105]:
import os
import fnmatch
def find_files(topdir, pattern):
    for path, dirname, filelist in os.walk(topdir):
        for name in filelist:
            if fnmatch.fnmatch(name, pattern):
                yield os.path.join(path,name)

def opener(filenames):
    for name in filenames:
        f = open(name)
        yield f
        
def cat(filelist):
    for f in filelist:
        for line in f:
            yield line
            
def grep(pattern, filelist):
    for line in lines:
        if pattern in line:
            yield line

In [109]:
passwd = find_files(".", "passwd")
files = opener(passwd)
lines = cat(files)
pylines = grep("linux", lines)
for line in pylines:
    print(line)

linux:x:1000:1000:linux,,,:/home/linux:/bin/bash



In [113]:
import os
import fnmatch

@coroutine
def find_files(target):
    while True:
        topdir, pattern = (yield)
        for path, dirname, filelist in os.walk(topdir):
            for name in filelist:
                if fnmatch.fnmatch(name, pattern):
                    target.send(os.path.join(path,name))

@coroutine                
def opener(target):
    while True:
        name = (yield)
        f = open(name)
        target.send(f)
        
@coroutine                
def cat(target):
    while True:
        f = (yield)
        for line in f:
            target.send(line)

@coroutine                
def grep(pattern, target):
    while True:
        line = (yield)
        if pattern in line:
            target.send(line)
            
@coroutine                
def printer():
    while True:
        line = (yield)
        print(line)

In [116]:
finder = find_files(opener(cat(grep("linux", printer()))))
finder.send((".", "passwd"))

linux:x:1000:1000:linux,,,:/home/linux:/bin/bash

